### 4.3 Tensorflow Project - Mnist CNN with Tensorboard

https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard
"""A simple MNIST classifier which displays summaries in TensorBoard.
This is an unimpressive MNIST model, but it is a good example of using
tf.name_scope to make a graph legible in the TensorBoard graph explorer, and of
naming summary tags so that they are grouped meaningfully in TensorBoard.
It demonstrates the functionality of every TensorBoard dashboard.
"""

### 1. import the lib and set the configuration

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

FLAGS = None
data_dir = './tmp'
log_dir = './tmp/log/mnist/'
EXPORT_PB_MODEL_VERSION = '1'
exportPBModelPath = './ncModel/'+EXPORT_PB_MODEL_VERSION

fake_data = True
learning_rate=0.001
max_steps = 200
dropout=0.4

Instructions for updating:
Use the retry module or similar alternatives.


### 2. import the data

In [2]:
# Import data
mnist = input_data.read_data_sets(data_dir, fake_data=fake_data)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### 3. create the multilayer tensorflow model

In [3]:
# We can't initialize these variables to 0 - the network will get stuck.
def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)

def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses ReLU to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable([input_dim, output_dim])
        variable_summaries(weights)
      with tf.name_scope('biases'):
        biases = bias_variable([output_dim])
        variable_summaries(biases)
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.summary.histogram('pre_activations', preactivate)
      activations = act(preactivate, name='activation')
      tf.summary.histogram('activations', activations)
      return activations

In [4]:
sess = tf.InteractiveSession()
# Create a multilayer model.

# Input placeholders
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.int64, [None], name='y-input')

with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

hidden1 = nn_layer(x, 784, 500, 'layer1')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

# Do not apply softmax activation yet, see below.
y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

with tf.name_scope('cross_entropy'):
    # The raw formulation of cross-entropy,
    #
    # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
    #                               reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.losses.sparse_softmax_cross_entropy on the
    # raw logit outputs of the nn_layer above, and then average across
    # the batch.
    with tf.name_scope('total'):
      cross_entropy = tf.losses.sparse_softmax_cross_entropy(
          labels=y_, logits=y)
tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(
        cross_entropy)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), y_)
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries and write them out to
# /tmp/tensorflow/mnist/logs/mnist_with_summaries (by default)
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test')
tf.global_variables_initializer().run()
print("log will be written to", log_dir + 'train')
print("log will be written to", log_dir + 'test')


log will be written to ./tmp/log/mnist/train
log will be written to ./tmp/log/mnist/test


In [5]:
# Train the model, and also write summaries.
# Every 10th step, measure test-set accuracy, and write test summaries
# All other steps, run train_step on training data, & add training summaries

def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train or fake_data:
      xs, ys = mnist.train.next_batch(100, fake_data=fake_data)
      k = dropout
    else:
      xs, ys = mnist.test.images, mnist.test.labels
      k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

for i in range(max_steps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
      if i % 100 == 99:  # Record execution stats
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step],
                              feed_dict=feed_dict(True),
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        train_writer.add_summary(summary, i)
        print('Adding run metadata for', i)
      else:  # Record a summary
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
        
train_writer.close()
test_writer.close()

Accuracy at step 0: 0.0
Accuracy at step 10: 1.0
Accuracy at step 20: 1.0
Accuracy at step 30: 1.0
Accuracy at step 40: 1.0
Accuracy at step 50: 1.0
Accuracy at step 60: 1.0
Accuracy at step 70: 1.0
Accuracy at step 80: 1.0
Accuracy at step 90: 1.0
Adding run metadata for 99
Accuracy at step 100: 1.0
Accuracy at step 110: 1.0
Accuracy at step 120: 1.0
Accuracy at step 130: 1.0
Accuracy at step 140: 1.0
Accuracy at step 150: 1.0
Accuracy at step 160: 1.0
Accuracy at step 170: 1.0
Accuracy at step 180: 1.0
Accuracy at step 190: 1.0
Adding run metadata for 199
Accuracy at step 200: 1.0
Accuracy at step 210: 1.0
Accuracy at step 220: 1.0
Accuracy at step 230: 1.0
Accuracy at step 240: 1.0
Accuracy at step 250: 1.0
Accuracy at step 260: 1.0
Accuracy at step 270: 1.0
Accuracy at step 280: 1.0
Accuracy at step 290: 1.0
Adding run metadata for 299
Accuracy at step 300: 1.0
Accuracy at step 310: 1.0
Accuracy at step 320: 1.0
Accuracy at step 330: 1.0
Accuracy at step 340: 1.0
Accuracy at step 3

### Tensorboard
steps:
- Terminal: tensorboard logdir='./tmp/log'
- IE: http://localhost:6006/
<img src='images/tensorboardScalar.png' style='width:1024px'>

### export/view the pb files 

In [ ]:
# tf.train.write_graph(self.sess.graph_def, self.exportPBModelPath, self.exportPBModelName, False)
print("tf.train.write_graph will save model to pb file{0}......".format(exportPBModelPath))
builder = tf.saved_model.builder.SavedModelBuilder(exportPBModelPath)
# Build the signature_def_map.
serialized_tf_x = x
serialized_tf_y_ = y_

prediction_output_winner=self.output_winner
# prediction_outputs=self.outputs

prediction_inputs_des_coords = build_tensor_info(serialized_tf_des_coords)
prediction_inputs_des_values = build_tensor_info(serialized_tf_des_values)
prediction_inputs_des_dims = build_tensor_info(serialized_tf_des_dims)

prediction_inputs_sch_coords = build_tensor_info(serialized_tf_sch_coords)
prediction_inputs_sch_values = build_tensor_info(serialized_tf_sch_values)
prediction_inputs_sch_dims = build_tensor_info(serialized_tf_sch_dims)

prediction_inputs_att_coords = build_tensor_info(serialized_tf_att_coords)
prediction_inputs_att_values = build_tensor_info(serialized_tf_att_values)
prediction_inputs_att_dims = build_tensor_info(serialized_tf_att_dims)

prediction_outputs_winner = build_tensor_info(prediction_output_winner)
# prediction_outputs = build_tensor_info(prediction_outputs)

prediction_signature = (
    tf.saved_model.signature_def_utils.build_signature_def(
        inputs={
            'des_coords':prediction_inputs_des_coords,
            'des_values':prediction_inputs_des_values,
            'des_dims':prediction_inputs_des_dims,

            'sch_coords':prediction_inputs_sch_coords,
            'sch_values':prediction_inputs_sch_values,
            'sch_dims':prediction_inputs_sch_dims,

            'att_coords':prediction_inputs_att_coords,
            'att_values':prediction_inputs_att_values,
            'att_dims':prediction_inputs_att_dims
        },
        outputs={
            'output_winner': prediction_outputs_winner
            # ,'outputs': prediction_outputs
            },
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

builder.add_meta_graph_and_variables(
        session,
        [tag_constants.SERVING],
        signature_def_map={'predict_category':prediction_signature}
        )
builder.save()
print("Done Part Category Classifier model exporting ")